# RULES OF THE GAME

In [ ]:

"""
1) Equipment :
    3 red dice for the attack
    3 blue dice for the defense
    42 territory cards
    Troops (Nb of troops limited only for the game setup but not for the game play)
Not included in this game simulation :
    42Risk cards with symbols(Infantry,artillery,calvalry)
    14 mission cards: LATER
    No limit of troop

2) Preliminary set up of game
Choose of troops depending of number of players => 3 players => 35 troops....
Distribute all the territories randomly to the players and add exactly one troop of the player  on the territory.
For each player, assign his remaining troops randomly on the territories he owns.

For this simulation the number of players is 2.

3)Define who goes first? ==> assign a player number to eacher player

4)The game itself
It is a turn by turn game. Each player has 3 actions to do.
    It is divided in 3 phases:
        
        Action 1: reinforcement
The current player add extra troops to his territories. Therea are 2 ways to get troops to reinforce his army:
    - get on 1 troop for every 3 territories owned
    - get bonus troops if you own a whole continent. The bonus depends on the continent.
    
        Action 2: attack
The current player can attack as often as he wants.
The current player announces which territory will atttacks which territory. 
The current player has to be sure that:
    - the atttacking territory is among the territories he is occupying.
    - the 2 territories are actually neighbours.
    - the destination territory does not belong to the player 
    - and if the number of troops is higher than 1 due to the 'cannot empty territory rule'
The current player decides our many troops he wants to commit for the attack
    make sure that the number of troops attacking is <4 LATER
    
Both the attacking and the attacked players roll their dices (3 red dice for attacks, 3 blue dice for the defense) on both sides and compared the max of each side .
The max wins and the min loses one troop . 
Automatically roll the dices until there a strict difference between the dice.

The current player can keep attacking as long as he wants or until he cannot play 'loss or only territories with one troop'

Display the board after every action

Phase 3: Manoeuvre
The current player can move only one group of troops from one territory to another. 
The territories have to belong to the current player.
The manoevre has to respect the "cannot empty territory rule".
    
5) Engame condition
Either Player conquers at least 30 territories.
    
"""

# Librairies

In [311]:
import random
import matplotlib
import time
from IPython.display import clear_output

In [ ]:
%matplotlib inline

# Inputs

In [354]:
#dictionary: territory_id and its name     
territory_id={
    1: "Alaska", 2: "Alberta", 3: "Central America", 4: "Eastern United States", 5: "Greenland",
    6: "Northwest Territory", 7: "Ontario", 8: "Quebec", 9: "Western United States",10: "Argentina", 
    11: "Brazil", 12: "Peru", 13: "Venezuela", 14: "Great Britain", 15: "Iceland", 
    16: "Northern Europe", 17: "Scandinavia", 18: "Southern Europe", 19: "Ukraine", 20: "Western Europe",
    21: "Congo", 22: "East Africa", 23: "Egypt", 24: "Madagascar", 25: "North Africa",
    26: "South Africa", 27: "Afghanistan", 28: "China", 29: "India", 30: "Irkutsk",
    31: "Japan", 32: "Kamchatka", 33: "Middle East", 34: "Mongolia", 35: "Siam ",
    36: "Siberia", 37: "Ural", 38: "Yakutsk", 39: "Eastern Australia", 40: "Indonesia",
    41: "New Guinea",42: "Western Australia"
} 
    
# dictionary: territory_id and a list of its owners id with the number of troops occupied with
territory_occupation={
    1: ["", 0],2: ["", 0],3: ["", 0],4: ["", 0],5: ["", 0],6: ["", 0],7: ["", 0],8: ["", 0],9: ["", 0],10: ["", 0],11: ["", 0],12: ["", 0],
    13: ["", 0],14: ["", 0],15: ["", 0],16: ["", 0],17: ["", 0],18: ["", 0],19: ["", 0],20: ["", 0],21: ["", 0],22: ["", 0],23: ["", 0],
    24: ["", 0],25: ["", 0],26: ["", 0],27: ["", 0],28: ["", 0],29: ["", 0],30: ["", 0],31: ["", 0],32: ["", 0],33: ["", 0],
    34: ["", 0],35: ["", 0],36: ["", 0],37: ["", 0],38: ["", 0],39: ["", 0],40: ["", 0],41: ["", 0],42: ["", 0],
}
    
#Dictionnary: list of continents with the territories in each continent
continent_id={
    "North America":  [1, 2, 3, 4, 5, 6, 7, 8, 9],
    "South America": [10, 11, 12, 13],
    "Europe": [14, 15, 16, 17, 18, 19, 20],
    "Africa": [21, 22, 23, 24, 25, 26],
    "Asia": [27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38],
    "Australia": [39, 40, 41, 42]    
}
    
    #Dictionnary: list of the territories with the borders (neighbour territories)
neighbour_id={
    1: [2, 6, 32],
    2: [1, 6, 7, 9],
    9: [2, 3, 4, 7],
    10: [11, 12],
    11: [10, 12, 13, 25],
    12: [10, 11, 13],
    13: [3, 11, 12],
    14: [15, 16, 17, 20],
    15: [5, 16, 17],
    16: [14, 17, 18, 19, 20],
    17: [14, 15, 16, 19],
    18: [16, 19, 20, 23, 25, 33],
    19: [16, 17, 27, 33, 37],
    20: [14, 16, 18, 25],
    21: [22, 25, 26],
    22: [21, 23, 24, 25, 26, 33],
    23: [18, 22, 25, 33],
    24: [22, 26],
    25: [11, 18, 20, 21, 22, 23],
    26: [21, 22, 24],
    27: [19, 28, 29, 33, 37],
    28: [27, 29, 34, 35, 36, 37],
    29: [27, 28, 33, 35],
    30: [32, 34, 36, 38],
    31: [32, 34],
    32: [1, 30, 31, 34, 38],
    33: [18, 19, 22, 23, 27, 29],
    34: [28, 30, 31, 32, 36],
    35: [28, 29, 40],
    36: [28, 30, 34, 37, 38],
    37: [19, 27, 28, 36],
    38: [30, 32, 36],  
    39: [41, 42],
    40: [35, 41, 42],
    41: [39, 40, 42],
    42: [39, 40, 41]
}
  
continent_bonus={
    "North America":  3,
    "South America": 1,
    "Europe": 2,
    "Africa": 2,
    "Asia": 5,
    "Australia": 1    
}

# Player Class

In [337]:
class Player:
    
    #global attributes
    lst_player=[] # list of all the players
    test=""

    def __init__(self,id_player):
        self.name='' #name_player
        self.id=id_player  # position of the player in the game
        self.occupied_territories=dict() # dictionnary with the territories owned and the number of troops on each territory
        self.riskcards=[]#Later maybe
        self.missions=[] #Later maybe
        self.stock=0 #counter for the reinforcement phase
        self.color='' # color of the player : depends on the player id
        
     #provide the list of territories occupied by the player 
    def get_occupied_territories(self):
        self.occupied_territories={x:territory_occupation[x][1] for x in territory_occupation.keys() if territory_occupation[x][0]==self.id}
        return self.occupied_territories
   
    def continent_check(self):
        progress={i: set(j).issubset(self.occupied_territories.keys()) for i,j in continent_id.items()}
        return progress
  
    def count_nb_troop_to_place(self):
        #owned territory bonus
        bonus_nb_territories=int((len(list(self.occupied_territories.keys())))/3)
    
        #continent bonus
        owned_continent=self.continent_check().values()
        if sum(owned_continent)>0:
            for i in self.continent_check().keys():
                if self.continent_check()[i]==True:
                    bonus_continent+=continent_bonus[i]
        else:
            bonus_continent=0
        reinforcement_troop=bonus_nb_territories+ bonus_continent
        return reinforcement_troop
     
    def reinforce_troop(self):
        nb_troop_to_place=self.count_nb_troop_to_place()
        while nb_troop_to_place!=0: 
            territory=random.choice(list(self.occupied_territories.keys()))
            print(territory)
            self.occupied_territories[territory]+=self.occupied_territories[territory]
            territory_occupation[territory][1]+=1
            nb_troop_to_place-=1
            self.stock-=1

    def move(self, from_territory_id, to_territory_id, troops):
        '''
        Validity checks on:
        1) from_territory belongs to player
        2) selected troops are available (i.e. enough troops at from_territory and at least 1 is remain stationed)
        3) to_territory is a neighbour of from_territory
        4) to_territory owner is not player
        '''
        outcome = (from_territory_id in self.occupied_territories.keys(), territory_occupation[from_territory_id][1]-troops>=1, to_territory_id in neighbour_id[from_territory_id], to_territory_id not in self.occupied_territories.keys())
        return outcome
    
    def attack_m(self, from_territory_id, to_territory_id, troops):
        # Mechanism for an attack
        
        # Valid move check
        if sum(self.move(from_territory_id, to_territory_id, troops))==4:
            original_force=troops
            # Update territory_occupation dictionary to move troops out of from_territory
            territory_occupation[from_territory_id][1]-=original_force
            # loop until "last-man standing"
            while (troops>0) & (territory_occupation[to_territory_id][1]>0):
                attack, defence = self.roll_dice()
                print(f"Attacker: {troops} vs Defender: {territory_occupation[to_territory_id][1]}")
                time.sleep(1)
                print(f"Attacker: {attack} vs Defender: {defence}")
                time.sleep(1)
                if sum(attack)==sum(defence):
                    print(f"It's a draw, another battle begins.")
                elif sum(attack)>sum(defence):
                    print(f"Attacker kills one troop of the defender.")
                    territory_occupation[to_territory_id][1]-=1               
                else:
                    print(f"Attacker lost one troop.")
                    troops-=1
                time.sleep(1)
                
            # check if attempted conquer was a success    
            if troops>0:
                self.get_occupied_territories()
                territory_occupation[to_territory_id]=[self.id, troops]
                print(f"Successfully conquered {territory_id[to_territory_id]}, moving in with {troops}", "troops." if troops>1 else "troop.")
                
                if self.win_check():
                    print("You have won the game.")
                else:
                    pass
            
            else:
                print(f"Failed attempt to conquer {territory_id[to_territory_id]}, {original_force}", "troops were" if original_force>1 else "troop was", "killed.")
            self.get_occupied_territories() 
        else:
            print("criteria not met for an attack")
    
    def attack(self):
        # Update player's occupied_territories dictionary
        self.get_occupied_territories()
        attack_on = True
        while attack_on:
            print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
            if input('Would you like to make any attacks? Y/N ').upper() == "Y":
                clear_output()
                attack_input=True
                while attack_input:
                    try:
                        print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
                        self.attack_m(self.convert_territory_id(input(f"""Please enter the territory you would want to attack from: """).title()), self.convert_territory_id(input('Please enter the territory you would want to attack: ').title()), int(input('Enter the number of troops to deploy for this attack: ')))
                    except:
                        print("Please make sure you have enter the information in the correct format.")
                    else:
                        break
            else:
                attack_on=False

    def roll_dice(self):
        red_dice, blue_dice=random.choices(range(1,7), k=3), random.choices(range(1,7), k=3)
        return red_dice, blue_dice

    def manoeuvre_m(self, from_territory_id, to_territory_id, troops):
        # Mechanism for a manoeuvre
        
        # Valid move check
        if sum(self.move(from_territory_id, to_territory_id, troops)[:3])==3 & sum(self.move(from_territory_id, to_territory_id, troops))!=4:
            territory_occupation[to_territory_id][1]+=troops
            territory_occupation[from_territory_id][1]-=troops
            print(f"You have moved {troops}",  "troops" if troops>1 else "troop", f"into {territory_id[to_territory_id]}.")
            self.get_occupied_territories()
        else:
            print("criteria not met for a manoeuvre")
    
    def manoeuvre(self):
        # Phase 3: Manoeuvre, ask obtain player input
        # Update player's occupied territories dictionary
        self.get_occupied_territories()
        manoeuvre_on = True
        while manoeuvre_on:
            print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
            if input('Would you like to make any manoeuvre? Y/N ').upper() == "Y":
                clear_output()
                manoeuvre_input=True
                while manoeuvre_input:
                    try:
                        print(f"""Current territories:
{[territory_id[i] for i,j in self.occupied_territories.items()]}""")
                        self.manoeuvre_m(self.convert_territory_id(input(f"""Please enter the territory you would want to manoeuvre from: """).title()), self.convert_territory_id(input('Please enter the territory you would want to manoeuvre to: ').title()), int(input('Enter the number of troops to deploy for this manoeuvre: ')))
                    except:
                        print("Please make sure you have enter the information in the correct format.")
                    else:
                        break
            else:
                manoeuvre_on=False
    
    def win_check(self):
        return len(self.occupied_territories)>=30
        pass
    
    def convert_territory_id(self, territory):
        return [i for i,j in territory_id.items() if j==territory][0]  
        

In [338]:
Joueur1=Player(1)
Joueur2=Player(2)

# WHO GOES FIRST?

In [339]:

#Who goes first? ==> give a player number no the players 
lst_player=['Alice','Bob']
random.shuffle(lst_player)
Joueur1.name=lst_player[0]
Joueur2.name=lst_player[1]
print(Joueur1.name,Joueur2.name)

Alice Bob


In [340]:


#for 2 players / to be adapted for more players
Nb_player=2
#choose of Number of troops in each game depending on the number of players
def game_setup(NbPlayer):
    army_size={2:40,3:35,4:30,5:35,6:20}
    return army_size[NbPlayer]

def assign_territories(Nb_player):
    #shuffle the territories
    deck_of_territories=random.sample(territory_id.keys(),len(territory_id.keys()))

    #distribute all the territories randomly to the players and add exactly one troop of the player  on the territory.

    for i,territory in enumerate(deck_of_territories):
        player=Joueur1
        if i%2==0:
            player=Joueur2   
        player.occupied_territories[territory]=1
        #Update the territory_occupation
        territory_occupation[territory]=[player.id,1]
    #Update stock of remaining Troops for each player
    Joueur1.stock=game_setup(Nb_player)-len(list(Joueur1.occupied_territories.keys()))
    Joueur2.stock=game_setup(Nb_player)-len(list(Joueur2.occupied_territories.keys())) 
    #dispatch remaining troops randomly on the occupied territories for each player
    while Joueur1.stock!=0 or Joueur2.stock!=0: 
        for player in [Joueur1, Joueur2]:
            if player.stock != 0:
                territory=random.choice(list(player.occupied_territories.keys()))
                player.occupied_territories[territory]+=player.occupied_territories[territory]
                territory_occupation[territory][1]+=1
                player.stock -=1
    return 


assign_territories(Nb_player)
print('J2 stock',Joueur2.stock)
print('J1 stock',Joueur1.stock)
print('J2  ',Joueur2.occupied_territories.keys())
print('J1 ',Joueur1.occupied_territories.keys())
print('TOCC' , territory_occupation)


J2 stock 0
J1 stock 0
J2   dict_keys([29, 19, 22, 23, 25, 2, 35, 39, 9, 15, 18, 36, 10, 33, 28, 40, 31, 30, 12, 41, 24])
J1  dict_keys([1, 16, 4, 8, 42, 14, 17, 3, 26, 38, 20, 5, 32, 7, 34, 27, 6, 21, 37, 13, 11])
TOCC {1: [1, 2], 2: [2, 1], 3: [1, 2], 4: [1, 2], 5: [1, 2], 6: [1, 2], 7: [1, 1], 8: [1, 5], 9: [2, 1], 10: [2, 1], 11: [1, 2], 12: [2, 2], 13: [1, 2], 14: [1, 2], 15: [2, 1], 16: [1, 2], 17: [1, 2], 18: [2, 1], 19: [2, 4], 20: [1, 2], 21: [1, 1], 22: [2, 3], 23: [2, 1], 24: [2, 1], 25: [2, 2], 26: [1, 4], 27: [1, 2], 28: [2, 5], 29: [2, 3], 30: [2, 2], 31: [2, 2], 32: [1, 1], 33: [2, 3], 34: [1, 1], 35: [2, 2], 36: [2, 1], 37: [1, 1], 38: [1, 1], 39: [2, 1], 40: [2, 2], 41: [2, 1], 42: [1, 1]}


In [348]:
Joueur1.reinforce_troop()

3
16
16
26
27
32
37


In [349]:
print(Joueur1.occupied_territories)
print(territory_occupation)

{1: 2, 16: 8, 4: 2, 8: 16, 42: 1, 14: 2, 17: 2, 3: 4, 26: 16, 38: 1, 20: 2, 5: 4, 32: 4, 7: 2, 34: 1, 27: 4, 6: 2, 21: 2, 37: 2, 13: 2, 11: 16}
{1: [1, 2], 2: [2, 1], 3: [1, 3], 4: [1, 2], 5: [1, 3], 6: [1, 2], 7: [1, 2], 8: [1, 5], 9: [2, 1], 10: [2, 1], 11: [1, 5], 12: [2, 2], 13: [1, 2], 14: [1, 2], 15: [2, 1], 16: [1, 4], 17: [1, 2], 18: [2, 1], 19: [2, 4], 20: [1, 2], 21: [1, 2], 22: [2, 3], 23: [2, 1], 24: [2, 1], 25: [2, 2], 26: [1, 5], 27: [1, 3], 28: [2, 5], 29: [2, 3], 30: [2, 2], 31: [2, 2], 32: [1, 3], 33: [2, 3], 34: [1, 1], 35: [2, 2], 36: [2, 1], 37: [1, 2], 38: [1, 1], 39: [2, 1], 40: [2, 2], 41: [2, 1], 42: [1, 1]}


In [351]:
Joueur1.attack_m(32,39,1)

criteria not met for an attack


In [ ]:
"""
 
risk_cards={
    1, "infantry", 2: "artillery", 3: "cavalry", 4: "infantry", 5: "artillery",
    6: "cavalry", 7: "infantry", 8: "artillery", 9: "cavalry",10: "infantry", 
    11: "artillery", 12: "cavalry", 13: "cavalry", 14: "artillery", 15: "cavalry", 
    16: "cavalry", 17: "artillery", 18: "infantry", 19: "artillery", 20: "cavalry",
    21: "infantry", 22: "artillery", 23: "cavalry", 24: "infantry", 25: "artillery",
    26: "cavalry", 27: "infantry", 28: "artillery", 29: "cavalry", 30: "infantry",
    31: "artillery", 32: "cavalry", 33: "infantry", 34: "artillery", 35: "cavalry ",
    36: "infantry", 37: "artillery", 38: "cavalry", 39: "infantry", 40: "artillery",
    41: "cavalry",42: "infantry"
}

Bonus_risk_card_combination={"infantry":4, "cavalry": 6, "artillery": 8, "OneofEach":10}"""